In [1]:
# Cell 1: Imports
import itertools
import numpy as np
import pandas as pd

import pyomo.environ as pyo
from pyomo.environ import value
from SOCF import run_self_optimizing_model
#from SOCF_3 import run_self_optimizing_model

In [2]:
# Cell 2: Base‐Model Constructor
def build_base_model():
    """
    Returns a Pyomo ConcreteModel with:
    - Variables: Xa, Xb, Xc in [0,1], M ≥ 0
    - Params: Kb, F, Zb, Za (mutable), Ka
    - Mass‐balance constraints eq1, eq2, eq3
    - Objective J = 100*Xb (maximize)
    """
    m = pyo.ConcreteModel()

    # Parameters
    m.X1   = pyo.Param(initialize=5.0, mutable=True)    # Feed solids (%)
    m.T1   = pyo.Param(initialize=40.0, mutable=True)   # Feed temperature (°C)
    m.T200 = pyo.Param(initialize=25.0, mutable=True)   # Cooling water inlet (°C)



    # Variables
    m.P2 = pyo.Var(bounds=(40,80), initialize= 51.412)
    m.T2 = pyo.Var(initialize= 88.400)
    m.T3 = pyo.Var(initialize= 81.066)
    m.F2 = pyo.Var(bounds=(0,None), initialize= 1.334)
    m.F100 = pyo.Var(bounds=(0,None), initialize= 9.434)
    m.T201 = pyo.Var(initialize= 45.550)
    m.F3 = pyo.Var(bounds=(0,100), initialize= 24.721)
    m.F5 = pyo.Var(initialize= 8.135)
    m.F200 = pyo.Var(bounds=(0,400), initialize= 217.738)
    m.F1 = pyo.Var(bounds=(0,20), initialize= 9.469)
    m.P100  = pyo.Var(bounds=(400, 400), initialize=400)
    m.F4   = pyo.Var(initialize=8.13)
    m.X2   = pyo.Var(bounds=(0, 100), initialize=35.5)
    m.T100 = pyo.Var(initialize=151.52)
    m.Q100 = pyo.Var(initialize=345.292)
    m.Q200 = pyo.Var(initialize=313.21)
                      
    # Constraints
    
   
    m.eq7 = pyo.Constraint(expr= m.F1 - m.F4 - m.F2 == 0)
    m.eq8 = pyo.Constraint(expr= m.F1*m.X1/100 - m.F2*m.X2/100 == 0)
    m.eq9 = pyo.Constraint(expr= m.F4 - m.F5 == 0)
    m.eq10 = pyo.Constraint(expr= m.T2   == 0.5616*m.P2 + 0.3126*m.X2 + 48.43)
    m.eq11 = pyo.Constraint(expr= m.T3   == 0.507*m.P2 + 55.0)
    m.eq12 = pyo.Constraint(
        expr= m.F4 == (m.Q100 - 0.07*m.F1*(m.T2 - m.T1)) / 38.5
    )
    m.eq13 = pyo.Constraint(expr= m.T100 == 0.1538*m.P100 + 90.0)
    m.eq14 = pyo.Constraint(
        expr= m.Q100 == 0.16*(m.F1 + m.F3)*(m.T100 - m.T2)
    )
    m.eq15 = pyo.Constraint(expr= m.F100 == m.Q100 / 36.6)
    m.eq16 = pyo.Constraint(
        expr= m.Q200 == 0.9576*m.F200*(m.T3 - m.T200)/(0.14*m.F200 + 6.84)
    )
    m.eq17 = pyo.Constraint(
        expr= m.T201 == m.T200
                         + 13.68*(m.T3 - m.T200)/(0.14*m.F200 + 6.84)
    )
    m.eq18 = pyo.Constraint(expr= m.F5   == m.Q200 / 38.5)

    m.spec    = pyo.Constraint(expr= m.X2 >= 35.5)

    # Bounds as separate constraints
    m.press_lb = pyo.Constraint(expr= m.P2    >= 40)
    m.press_ub = pyo.Constraint(expr= m.P2    <= 80)
    m.f200_lb  = pyo.Constraint(expr= m.F200  >= 0)
    m.f200_ub  = pyo.Constraint(expr= m.F200  <= 400)
    m.f1_lb    = pyo.Constraint(expr= m.F1    >= 0)
    m.f1_ub    = pyo.Constraint(expr= m.F1    <= 20)
    m.f3_lb    = pyo.Constraint(expr= m.F3    >= 0)
    m.f3_ub    = pyo.Constraint(expr= m.F3    <= 100)
    m.P100_ul  = pyo.Constraint(expr= m.P100  <= 400)

    # Objective
    m.J = pyo.Objective(expr= (600*m.F100 + 0.6*m.F200 + 1.009*(m.F2 + m.F3) + 0.2*m.F1 - 4800*m.F2), sense= pyo.minimize)

    return m

m = build_base_model()
import pprint
pprint.pprint(m.J.sense)


<ObjectiveSense.minimize: 1>


In [ ]:
disturbances = {
    "X1":  [5.0, 5.0 * 0.95, 5.0 * 1.05],
    "T1": [40.0, 40.0 * 0.80, 40.0 * 1.200],
    "T200": [25.0,25.0 * 0.80, 25.0 * 1.20],
}

Controlled_Variables = {
        
    "F3= 24.72, F200= 217.7395":      lambda m: [m.F3.fix(24.72),m.F200.fix(217.739)],
    "T201= 45.5495,F3= 24.72":      lambda m: [m.T201.fix(45.5495), m.F3.fix(24.72)],
    "P2 = 51.411, T201= 45.5495": lambda m: [m.P2.fix(51.411),m.T201.fix(45.5495)],
    "F100=9.43, F200 = 217.739":      lambda m: [m.F100.fix(9.43),m.F200.fix(217.739)],
    "P2 = 51.411, F200 = 217.739": lambda m: [m.P2.fix(51.411),m.F200.fix(217.739)],
}

CVs = {
        "J":     lambda m, c: value(m.J),
        "F1":    lambda m, c: value(m.F1),
        "F2":    lambda m, c: value(m.F2),
        "F3":    lambda m, c: value(m.F3),
        "F5":    lambda m, c: value(m.F5),
        "F100":  lambda m, c: value(m.F100),
        "F200":  lambda m, c: value(m.F200),
        "P2":    lambda m, c: value(m.P2),
        "T201":    lambda m, c: value(m.T201),
}

In [4]:
if __name__ == "__main__":
    results_df, loss_matrix = run_self_optimizing_model(
        build_base_model,
        disturbances,
        Controlled_Variables,
        CVs
    )

    print("=== Optimal CVs under each disturbance ===")
    display(results_df)     
    print("\n=== Loss for alternative controlled variables ===")
    display(loss_matrix)

=== Optimal CVs under each disturbance ===


,Nominal,T200=20.0,T200=30.0,T1=32.0,T1=48.0,X1=4.75,X1=5.25
J,-582.2332,-647.4719,-520.3239,-497.9681,-672.8589,-258.1955,-1006.9959
F1,9.4690,10.0064,8.9303,8.7792,10.1496,7.7136,12.2889
F2,1.3337,1.4094,1.2578,1.2365,1.4295,1.0321,1.8174
F3,24.7211,24.5004,24.8841,21.4293,28.3955,17.4479,46.2598
F5,8.1353,8.5971,7.6725,7.5427,8.7201,6.6815,10.4715
F100,9.4342,9.9192,8.9423,8.8197,10.0271,7.6479,12.4894
F200,217.7393,229.6143,205.8379,201.3674,234.0725,144.9983,286.3533
P2,51.4117,46.7185,56.0878,44.8836,57.8446,40.0000,76.9172
T201,45.5495,40.5927,50.5010,45.6016,45.4896,50.3440,45.1127



=== Loss for alternative controlled variables ===


,"Loss with F3= 24.72, F200= 217.7395","Loss with T201= 45.5495,F3= 24.72","Loss with P2 = 51.411, T201= 45.5495","Loss with F100=9.43, F200 = 217.739","Loss with P2 = 51.411, F200 = 217.739"
Nominal,0.0,0.0,0.0,0.0,0.0
T200=20.0,0.36,6.69,6.49,1.36,1.66
T200=30.0,0.35,10.21,9.59,1.36,1.67
T1=32.0,2.51,2.42,3.19,2.11,3.19
T1=48.0,2.48,2.39,3.17,2.12,3.17
X1=4.75,40.87,40.83,41.16,41.27,41.16
X1=5.25,53.22,53.19,53.52,53.92,53.52
Average loss,14.26,16.53,16.73,14.59,14.91
Ranking,1.0,4.0,5.0,2.0,3.0
